# Project -  Predicting the stock Markets

## Introduction

### Preliminary

This Notebook is the conclusion of the **Machine Learning in Python: Intermediate course ** course from [dataquest.io](dataquest.io). It is a guided project whose aim is to use all the techniques and skills learnt during the course. We will be working here with data from the [S&P500 Index](https://en.wikipedia.org/wiki/S%26P_500).

Indexes aggregate the prices of multiple stocks together, and allow us to see how the market as a whole performs.  
The S&P500 Index aggregates the stock prices of 500 large companies. When an index fund goes up or down, you can say that the primary market or sector it represents is doing the same. For example, if the Dow Jones Industrial Average price goes down one day, you can say that American stocks overall went down (ie, most American stocks went down in price).


We'll be using historical data on the price of the S&P500 Index to make predictions about future prices. Predicting whether an index goes up or down helps forecast how the stock market as a whole performs. Since stocks tend to correlate with how well the economy as a whole is performs, it can also help with economic forecasts. 

#### Blockquotes usage
> I am sometimes using blockquotes as this one, meaning that for the rest of the project I am quoting some elements given by dataquest. For the sake of simplicity and clarity, I estimated that they did not need any reformulation and were immediately usable and convenient for me and the reader.

### Context



The dataset is csv file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in `sphist.csv.`

The columns of the dataset are:

    Date -- The date of the record.
    Open -- The opening price of the day (when trading starts).
    High -- The highest trade price during the day.
    Low -- The lowest trade price during the day.
    Close -- The closing price for the day (when trading is finished).
    Volume -- The number of shares traded.
    Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions. Read more here.

We'll be using this dataset to develop a predictive model. We'll train the model with data from 1950-2012 and try to make predictions from 2013-2015.


## Reading the Data

In [1]:
import pandas as pd
from datetime import datetime
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
data = pd.read_csv("sphist.csv")
data["Date"] = pd.to_datetime(data["Date"],format='%Y-%m-%d')
data = data.sort_values("Date",ascending=True)

In [2]:
data.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999


## Adding some columns

> Datasets taken from the stock market need to be handled differently than datasets from other sectors when it's time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential and each observation comes a day after the previous observation. Thus, the observations are not all independent and you can't treat them as such.

> This means you have to be extra careful **not to inject "future" knowledge into past rows** when you train and predict. Injecting future knowledge makes our model look good when we train and test it, but it fails in the real world. This is how many algorithmic traders lose money.

> The time series nature of the data means that we can generate indicators to make our model more accurate. For instance, you can create a new column that contains the average price of the last 10 trades for each row. This incorporates information from multiple prior rows into one and makes predictions much more accurate.

> When you do this, you have to be careful not to use the current row in the values you average. You want to teach the model how to predict the current price from historical prices. If you include the current price in the prices you average, it will be equivalent to handing the answers to the model upfront, and will make it impossible to use in the "real world", where you don't know the price upfront.

with the given indication and in order to make then our predictions more accurate, we are going to generate several indicators :
- average price of the last 5 days
- average for of the last 365 days
- ratio bewteen the average value of the last 5 days and the average value of the last 365 days
- standard deviation of the last 5 days
- standard deviation of the last 365 days
- ratio between the latters

In [3]:
data['avg_5_d'] = data.iloc[:,4].rolling(window=5).mean()
data['avg_365_d'] = data.iloc[:,4].rolling(window=365).mean()
data['std_5_d'] = data.iloc[:,4].rolling(window=5).std()
data['std_365_d'] = data.iloc[:,4].rolling(window=365).std()


In [4]:
data.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5_d,avg_365_d,std_5_d,std_365_d
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,16.900,NaN,0.157956,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.974,NaN,0.089051,NaN
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,17.022,NaN,0.067602,NaN
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,16.988,NaN,0.134796,NaN
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.926,NaN,0.196545,NaN
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.854,NaN,0.191912,NaN


We need to shift all the values "forward" one day for all the newly created columns as the rolling mean/std use the current day's price and we don't want to ** inject "future" knowledge into past rows** 

In [5]:
data[['avg_5_d','avg_365_d','std_5_d','std_365_d']] = data[['avg_5_d','avg_365_d','std_5_d','std_365_d']].shift(periods=1)

In [6]:
data.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5_d,avg_365_d,std_5_d,std_365_d
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,NaN,0.157956,NaN
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,NaN,0.089051,NaN
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,NaN,0.067602,NaN
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,NaN,0.134796,NaN
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,NaN,0.196545,NaN


In [7]:
data['ratio_avg_365_5'] = data['avg_5_d'] / data['avg_365_d']
data['ratio_std_365_5'] = data['std_5_d'] / data['std_365_d']


In [10]:
# We are dropping all the rows before a certain date and those containing Nan
data = data[data["Date"]>datetime(year=1951,month=1,day=2)]
data = data.dropna(axis=0)

We are splitting our original dataset into two datasets :
- train data (all the rows before January 1st 2013)
- test data (all the rows after January 1st 2013)

In [19]:
train = data[data["Date"]<datetime(year=2013,month=1,day=1)]
test = data[data["Date"]>=datetime(year=2013,month=1,day=1)]

In [20]:
train.tail(5)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5_d,avg_365_d,std_5_d,std_365_d,avg_5_v,avg_365_v,std_5_v,std_365_v,ratio_avg_365_5,ratio_std_365_5,ratio_v_avg_365_5,ratio_v_std_365_5
743,2012-12-24,1430.150024,1430.150024,1424.660034,1426.660034,1.248960e+09,1426.660034,1437.360010,1326.114028,7.622009,89.830647,4.108678e+09,3.886189e+09,6.990482e+08,7.626629e+08,1.083889,0.084849,1.057251,0.916589
742,2012-12-26,1426.660034,1429.420044,1416.430054,1419.829956,2.285030e+09,1419.829956,1436.620019,1326.412494,8.589693,89.983530,3.667348e+09,3.878488e+09,1.477523e+09,7.749962e+08,1.083087,0.095458,0.945561,1.906491
741,2012-12-27,1419.829956,1422.800049,1401.800049,1418.099976,2.830180e+09,1418.099976,1431.228003,1326.716494,9.058684,90.111444,3.263906e+09,3.872807e+09,1.535105e+09,7.790566e+08,1.078775,0.100528,0.842775,1.970467
740,2012-12-28,1418.099976,1418.099976,1401.579956,1402.430054,2.426680e+09,1402.430054,1427.685986,1326.995836,10.208568,90.236516,3.055982e+09,3.868936e+09,1.502584e+09,7.807206e+08,1.075878,0.113131,0.789876,1.924612
739,2012-12-31,1402.430054,1426.739990,1398.109985,1426.189941,3.204330e+09,1426.189941,1419.434009,1327.261562,10.701861,90.315637,2.804002e+09,3.864302e+09,1.475799e+09,7.842493e+08,1.069446,0.118494,0.725617,1.881799


In [21]:
test.head(5)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5_d,avg_365_d,std_5_d,std_365_d,avg_5_v,avg_365_v,std_5_v,std_365_v,ratio_avg_365_5,ratio_std_365_5,ratio_v_avg_365_5,ratio_v_std_365_5
738,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1327.534055,9.820801,90.463948,2.399036e+09,3.861288e+09,7.366851e+08,7.846668e+08,1.068629,0.108560,0.621305,0.938851
737,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1327.908247,22.261321,90.738976,2.989764e+09,3.862480e+09,7.674944e+08,7.848543e+08,1.073714,0.245334,0.774053,0.977881
736,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1328.224877,26.274326,90.995857,3.298704e+09,3.859719e+09,7.224721e+08,7.831860e+08,1.079412,0.288742,0.854649,0.922478
735,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,1328.557617,27.945242,91.279049,3.417526e+09,3.859449e+09,6.733364e+08,7.833194e+08,1.086423,0.306152,0.885496,0.859594
734,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,1328.898603,16.453319,91.544368,3.593184e+09,3.858814e+09,4.153739e+08,7.836758e+08,1.095093,0.179731,0.931163,0.530033


## Making Predictions

To build and train our model, we will leave out all of the original columns (`Close, High, Low, Open, Volume, Adj Close, Date`). These all contain knowledge of the future that we don't want to feed the model. We will use the `Close` column as the target.

The error metric we will use here is the ***mean absolute error*** , it will show us how "close" we were to the price in intuitive terms.

In [22]:

features = ['avg_5_d','avg_365_d','std_5_d','std_365_d','ratio_avg_365_5','ratio_std_365_5']
lr = linear_model.LinearRegression() 
fit = lr.fit(train[features],train["Close"]) 
test_predictions = lr.predict(test[features])
test_mea = mean_absolute_error(test["Close"],test_predictions)
print(test_mea)


16.1311238212


###  Improving Error ?

We will try to improve our model that is reducing our error, by adding new features to our Dataset. As we did before, we will compute the moving average / standard deviation for the last 5 and 365 days for the **`volume`** column.

In [23]:
data = pd.read_csv("sphist.csv")
data["Date"] = pd.to_datetime(data["Date"],format='%Y-%m-%d')
data = data.sort_values("Date",ascending=True)

data['avg_5_d'] = data.iloc[:,4].rolling(window=5).mean()
data['avg_365_d'] = data.iloc[:,4].rolling(window=365).mean()
data['std_5_d'] = data.iloc[:,4].rolling(window=5).std()
data['std_365_d'] = data.iloc[:,4].rolling(window=365).std()

#adding 4 other indicators
data['avg_5_v'] = data.iloc[:,5].rolling(window=5).mean()
data['avg_365_v'] = data.iloc[:,5].rolling(window=365).mean()
data['std_5_v'] = data.iloc[:,5].rolling(window=5).std()
data['std_365_v'] = data.iloc[:,5].rolling(window=365).std()


data[['avg_5_d','avg_365_d','std_5_d','std_365_d','avg_5_v','avg_365_v','std_5_v','std_365_v']] = \
data[['avg_5_d','avg_365_d','std_5_d','std_365_d','avg_5_v','avg_365_v','std_5_v','std_365_v']].shift(periods=1)


data['ratio_avg_365_5'] = data['avg_5_d'] / data['avg_365_d']
data['ratio_std_365_5'] = data['std_5_d'] / data['std_365_d']
#adding 2 new ratios
data['ratio_v_avg_365_5'] = data['avg_5_v'] / data['avg_365_v']
data['ratio_v_std_365_5'] = data['std_5_v'] / data['std_365_v']

In [24]:
data.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5_d,avg_365_d,std_5_d,std_365_d,avg_5_v,avg_365_v,std_5_v,std_365_v,ratio_avg_365_5,ratio_std_365_5,ratio_v_avg_365_5,ratio_v_std_365_5
16589,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16584,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,NaN,0.157956,NaN,2046000.0,NaN,529650.828377,NaN,NaN,NaN,NaN,NaN
16583,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,NaN,0.089051,NaN,2226000.0,NaN,298043.620968,NaN,NaN,NaN,NaN,NaN
16582,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,NaN,0.067602,NaN,2374000.0,NaN,272084.545684,NaN,NaN,NaN,NaN,NaN
16581,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,NaN,0.134796,NaN,2458000.0,NaN,382452.611444,NaN,NaN,NaN,NaN,NaN
16580,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,NaN,0.196545,NaN,2722000.0,NaN,446172.612337,NaN,NaN,NaN,NaN,NaN


In [25]:
data = data[data["Date"]>datetime(year=1951,month=1,day=2)]
data = data.dropna(axis=0)

train = data[data["Date"]<datetime(year=2013,month=1,day=1)]
test = data[data["Date"]>=datetime(year=2013,month=1,day=1)]

In [26]:
features = ['avg_5_d','avg_365_d','std_5_d','std_365_d','ratio_avg_365_5','ratio_std_365_5',\
            'avg_5_v','avg_365_v','ratio_v_avg_365_5','std_5_v','std_365_v','ratio_v_std_365_5']
lr = linear_model.LinearRegression() 
fit = lr.fit(train[features],train["Close"]) 
test_predictions = lr.predict(test[features])
test_mea = mean_absolute_error(test["Close"],test_predictions)
print(test_mea)


16.0561784379


**Conclusion** : the indicators, we added here did not really improve our model. Indeed the mean absolute error just slightly decreased from 16.13 to 16.06.

## To go further ...

Accuracy would improve greatly by making predictions only one day ahead.  
For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.